<a href="https://colab.research.google.com/github/ofakduman/Lstm-forecasting/blob/main/lstm_forecasting_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv("all_data.csv")

### logcs nan olanlari at, record ID tekrarlanmasin
### Food name kolonunu kaldir
### Temperature Nan lari kaldir ve temprature kolonundaki logcs degerleri gibi olan degerleri kaldir deger int e cevriliyorsa kalsin
### Nanlari kaldir Aw deki
### Assumed kolonunu kaldir
### Conditions kolonunu kaldir
### Max.rate(logc.conc / h) kolonunu kaldir

In [ ]:
df['Organism'].unique()
df['Organism'].isna().sum()


In [ ]:
len(df['Food category'].unique())
df['Food category'].isna().sum()


In [ ]:
len(df['Food Name'].unique())
# df['Food Name'].isna().sum()

In [ ]:
df['Temperature (C)'].unique()
# df['Temperature (C)'].isna().sum()

In [ ]:
df['Aw'].unique()
df['Aw'].isna().sum()

In [ ]:
df['pH'].unique()
df['pH'].isna().sum()

In [ ]:
df['Assumed'].unique()
df['Assumed'].isna().sum()
# df.columns


In [ ]:
print(df['Max.rate(logc.conc / h)'].unique())
# df['Max.rate(logc.conc / h)'].isna().sum()


In [ ]:
df.replace("Not available", pd.NA, inplace=True)
len(df)


In [ ]:
df_dropped = df.drop(['Conditions', 'Assumed', 'Max.rate(logc.conc / h)', 'Food Name'], axis=1)
df_dropped.head()
len(df_dropped.dropna())

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Organism').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
df_dropped.dropna(inplace=True)

In [ ]:
len(df_dropped)
df_dropped


In [ ]:
cleaned_df = df_dropped.drop_duplicates(subset='Record ID', keep='first')

In [ ]:
def analyze_logcs(logcs_str):
    # Değerleri ; ile böl ve float'a çevir
    values = list(map(float, logcs_str.split(';')))

    # Zaman ve populasyon değerlerini ayır
    times = values[::2]
    populations = values[1::2]

    # Populasyon değerlerinin artıp artmadığını kontrol et
    increases = 0
    decreases = 0
    for i in range(1, len(populations)):
        if populations[i] > populations[i-1]:
            increases += 1
        elif populations[i] < populations[i-1]:
            decreases += 1

    # Toplam değer sayısı ve artma/azalma sayılarına göre bir sonuç döndür
    return {
        "total_values": len(populations),
        "increases": increases,
        "decreases": decreases
    }

# `Logcs` kolonunu analiz edelim
cleaned_df_copy = cleaned_df.copy()
results = cleaned_df['Logcs'].apply(analyze_logcs)

# Sonuçları yeni kolonlarda saklayalım
cleaned_df_copy['Total Values'] = results.apply(lambda x: x["total_values"])
cleaned_df_copy['Increases'] = results.apply(lambda x: x["increases"])
cleaned_df_copy['Decreases'] = results.apply(lambda x: x["decreases"])

In [ ]:
cleaned_df_copy[cleaned_df_copy["Record ID"] == "Lm_BfG_31"]



In [ ]:
a = "0;8.7;0.12;7.88;0.17;6.15;0.25;5.78;0.33;4.97;0.42;3.87;0.5;2.34;0.75;1.69;0.83;1.39"

a.split(";")

In [ ]:
import pandas as pd

# Örnek Series verisi
data = {"Logcs": "0;8.7;0.12;7.88;0.17;6.15;0.25;5.78;0.33;4.97;0.42;3.87;0.5;2.34;0.75;1.69;0.83;1.39"}
series = pd.Series(data)

# String'i noktalı virgül ile ayır
values = series['Logcs'].split(';')

# Değerlerin sayısını bul
number_of_points = len(values)

print("Zaman serisindeki nokta sayısı:", number_of_points)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import normaltest

# Veri setinizi yükleyin
# cleaned_df_copy = pd.read_csv('your_dataset.csv')  # Örnek veri seti yolu
# Sayısal sütunları seçin (örnek olarak sadece sayısal verileri içeren sütunlar listelenmiştir)
numeric_columns = ['Temperature (C)', 'Aw', 'pH', 'Total Values', 'Increases', 'Decreases']
for col in numeric_columns:
    # Sayısal olmayan verileri ve NaN değerleri temizle
    cleaned_data = cleaned_df_copy[col].apply(pd.to_numeric, errors='coerce').dropna()
    squared_data = cleaned_data ** 2

    # Histogramı çiz
    plt.figure(figsize=(10, 4))
    sns.histplot(cleaned_data, kde=True)
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

    # Temel İstatistiksel Özellikler
    print(f"\n{col} - Statistical Species:")
    print(f"Min: {cleaned_data.min()}")
    print(f"Max: {cleaned_data.max()}")
    print(f"Mean: {cleaned_data.mean()}")
    print(f"Median: {cleaned_data.median()}")




In [ ]:
cleaned_df_copy[cleaned_df_copy['Record ID'] == 'PS203']

In [ ]:
trashold_sample_value = 5
df = cleaned_df_copy[cleaned_df_copy['Total Values'] > trashold_sample_value]
df = df[df['Temperature (C)'].apply(lambda x: ';' not in x)]
df = df[df['pH'].apply(lambda x: ';' not in x)]
df = df[df['Aw'].apply(lambda x: ';' not in x)]
len(df)

In [ ]:
def parse_logcs(logcs_str):
    pairs = logcs_str.split(';')
    parsed_pairs = []
    for i in range(0, len(pairs) - 1, 2):
        try:
            # Zaman değerini tam sayıya dönüştür
            time = float(pairs[i])
            # Değer olarak ondalık sayı kullan
            value = float(pairs[i + 1])
            parsed_pairs.append((time, value))
        except ValueError:
            # Geçersiz dönüşüm olduğunda bu çifti atla
            continue
    return parsed_pairs

df['Logcs'] = df['Logcs'].apply(parse_logcs)


In [ ]:
df.head()


In [ ]:
original_columns = set(df.columns)

df = pd.get_dummies(df, columns=['Organism', 'Food category'])

df.head()

In [ ]:
import pandas as pd


# Pandas'ın maksimum gösterim sınırlarını ayarlayın
pd.set_option('display.max_rows', 100)  # Satırları sınırsız göster
pd.set_option('display.max_columns', 100)  # Sütunları sınırsız göster

# DataFrame'i göster
df


In [ ]:
from sklearn.preprocessing import StandardScaler


features_to_scale = ['Temperature (C)', 'Aw', 'pH']
scaler = StandardScaler()

# Normalizasyonu sadece belirli sütunlara uygula
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])


In [ ]:
df[df["Record ID"]=="Lm_BfG_31_1"]

In [ ]:
import pandas as pd
import numpy as np


# Hariç tutulacak sütunlar
exclude_columns = ['Record ID','Total Values', 'Increases', 'Decreases','Logcs']

# all_features listesini oluşturma
all_features = [col for col in df.columns if col not in exclude_columns]

# Veriyi hazırlama
X = []  # Girdiler
y = []  # Hedefler

for index, row in df.iterrows():
    logcs_data = row['Logcs']
    # Tüm uygun özellikleri al
    features = row[all_features].values
    for i in range(len(logcs_data)-1):
        X.append(np.concatenate(([logcs_data[i][1]], features)))  # current value and other features
        y.append(logcs_data[i+1][1])  # next value

X = np.array(X).reshape(-1, 1, len(X[0]))  # (samples, time steps, features)
y = np.array(y).reshape(-1, 1)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Verilerinizi yükleyin veya oluşturun
# X ve y verilerini bu aşamada hazırlayın

# Verileri eğitim ve test kümesine bölün
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

# LSTM modeli oluşturun
model = keras.Sequential()
model.add(layers.LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(layers.LSTM(32, return_sequences=True))
model.add(layers.Dense(1))  # Çıkış katmanı

# Modeli derleyin
model.compile(optimizer='adam', loss='mean_squared_error')  # İstediğiniz bir optimizasyon ve loss fonksiyonu kullanabilirsiniz

# Early Stopping callback'i oluşturun
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Modeli eğitin ve Early Stopping callback'ini kullanın
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Modelin performansını değerlendirin
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

# Tahminler yapın
predictions = model.predict(X_test)

# Eğitim ve doğrulama kayıplarını görselleştirin
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

# Tahminleri ve gerçek değerleri alın
predictions = model.predict(X_test)

# Linear Regression modelini oluşturun ve eğitin
regression_model = LinearRegression()
y_test = y_test.reshape(-1, 1)  # Yeniden şekillendirme
predictions = predictions.reshape(-1, 1)  # Yeniden şekillendirme
regression_model.fit(y_test, predictions)

# Gerçek değerlerin minimum ve maksimum değerlerini alın
min_val = min(np.min(y_test), np.min(predictions))
max_val = max(np.max(y_test), np.max(predictions))

# Doğruyu çizmek için tahminler oluşturun
x_values = np.linspace(min_val, max_val, 100)
y_values = regression_model.predict(x_values.reshape(-1, 1))

# Scatter plot oluşturun
plt.figure(figsize=(10, 6))
plt.scatter(y_test, predictions, alpha=0.5)
plt.plot(x_values, y_values, color='red', linewidth=2, label='Regression Line')

# X=Y doğrusunu çizmek için
plt.plot([min_val, max_val], [min_val, max_val], color='blue', linewidth=2, label='X=Y Line')

plt.xlabel('Values')
plt.ylabel('Predictions')
plt.title('Values vs. Predictions')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
# Tahminlerin boyutunu kontrol edin ve düzeltin
predictions = np.squeeze(predictions)

# Ortalama Mutlak Hata (MAE)
mae = mean_absolute_error(y_test, predictions)
print(f"Mean Absolute Error (MAE): {mae:.3f}")

# Kök Ortalama Karesel Hata (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")

# R² Skoru
r2 = r2_score(y_test, predictions)
print(f"R² Score: {r2:.3f}")


In [ ]:
len(df)
df.head(1)

In [ ]:
from sklearn.model_selection import train_test_split

# df DataFrame'ini ilk olarak eğitim ve test setlerine ayır
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# df_train DataFrame'ini daha sonra eğitim ve doğrulama setlerine ayır
df_train_partial, df_val = train_test_split(df_train, test_size=0.2, random_state=42)


In [ ]:
df.head(1)

In [ ]:

def prepare_data(df):
  X, y = [], []
  exclude_columns = ['Record ID','Total Values', 'Increases', 'Decreases','Logcs']

  # all_features listesini oluşturma
  all_features = [col for col in df.columns if col not in exclude_columns]
  all_features


  for index, row in df.iterrows():
      logcs_data = row['Logcs']
      # all_features kullanarak özellikleri seç
      features = row[all_features].values

      if len(logcs_data) > 5:
          X_input = []
          for i in range(5):
              X_input.extend([logcs_data[i][1]])
              X_input.extend(features)

          y_output = [logcs_data[i][1] for i in range(5, len(logcs_data))]

          X.append(X_input)
          y.append(y_output)

  return np.array(X).reshape(-1, 5, len(X[0]) // 5), np.array(y, dtype=object)

# Modelin geri kalan kısmı aynı kalabilir
X_train, y_train = prepare_data(df_train_partial)
X_val, y_val = prepare_data(df_val)
X_test, y_test = prepare_data(df_test)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed

# Maksimum çıktı uzunluğunu hesapla
max_output_length = max(len(y) for y in y_train)

# Encoder
encoder_inputs = Input(shape=(5, len(X_train[0][0])))  # 5 zaman adımı, özellik sayısı
encoder_lstm = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = RepeatVector(max_output_length)(encoder_outputs)  # Burada encoder çıktısını tekrarla
decoder_lstm = LSTM(50, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = TimeDistributed(Dense(1))  # Her zaman adımı için bir tahmin
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model(encoder_inputs, decoder_outputs)

# Model Derleme
model.compile(optimizer='adam', loss='mean_squared_error')

# Model Özeti
model.summary()


In [ ]:
import tensorflow as tf
from sklearn.metrics import mean_squared_error

optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.MeanSquaredError()

epochs = 10  # Toplam epoch sayısı
train_losses = []
val_losses = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    total_train_loss = 0
    total_val_loss = 0

    # Eğitim döngüsü
    for i in range(len(X_train)):
        with tf.GradientTape() as tape:
            y_pred = model(X_train[i:i+1], training=True)
            train_loss = loss_fn(y_train[i], y_pred[0, :len(y_train[i]), 0])
        gradients = tape.gradient(train_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        total_train_loss += train_loss.numpy()

    # Doğrulama döngüsü
    for i in range(len(X_val)):
        # X_val[i:i+1] dizisini tensöre dönüştür
        X_val_tensor = tf.convert_to_tensor(X_val[i:i+1], dtype=tf.float32)
        y_pred_val = model.predict(X_val_tensor)

        # y_val[i] dizisini tensöre dönüştür
        y_val_tensor = tf.convert_to_tensor(y_val[i], dtype=tf.float32)

        # Tahmin ve gerçek değerlerin boyutlarını eşitle
        min_length = min(y_pred_val.shape[1], len(y_val[i]))
        val_loss = loss_fn(y_val_tensor[:min_length], y_pred_val[0, :min_length, 0])

        total_val_loss += val_loss.numpy()

    # Ortalama kayıpları hesapla ve kaydet
    avg_train_loss = total_train_loss / len(X_train)
    avg_val_loss = total_val_loss / len(X_val)
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)

    print(f"Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")

# Kayıpları grafik üzerinde görselleştirme
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Train & Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Test veri seti üzerinde tahmin yapma
y_pred = model.predict(X_test)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
def calculate_metrics(y_true, y_pred):
    mse_values = []
    mae_values = []
    r2_values = []

    for i in range(len(y_true)):
        true_values = np.array(y_true[i])
        pred_values = y_pred[i, :len(y_true[i]), 0]

        mse = mean_squared_error(true_values, pred_values)
        mae = mean_absolute_error(true_values, pred_values)

        mse_values.append(mse)
        mae_values.append(mae)

        # R2 skorunu yalnızca birden fazla değeri olan örnekler için hesap
        if len(true_values) > 1:
            r2 = r2_score(true_values, pred_values)
            r2_values.append(r2)

    return np.mean(mse_values), np.mean(mae_values), np.mean(r2_values) if r2_values else np.nan

# Hata metriklerini hesaplama
mse, mae, r2 = calculate_metrics(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R2 Score:", r2)


In [ ]:
import matplotlib.pyplot as plt
import random

selected_indices = random.sample(range(len(y_test)), 5)

for index in selected_indices:
    plt.figure(figsize=(8, 4))  # Grafik boyutu
    plt.plot(y_test[index], marker='o', label='y_test')

    # y_pred verisini y_test verisinin uzunluğu
    y_pred_trimmed = y_pred[index][:len(y_test[index])]
    plt.plot(y_pred_trimmed, marker='x', label='y_pred')

    plt.xlabel('Time Step')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.title(f'Sample {index + 1}')
    plt.show()

In [ ]:
df